# Génération de text à la façon de Guy de Maupassant.

<br>



<p align="center">
  <img width="700" height="400" src="https://github.com/AxelDucamp/PROJET_Text_Generation_Maupassant/blob/main/guy-de-maupassant.jpg?raw=true">
</p>


<br>
<br>

# Le projet :

Ce projet rapide a pour objectif de générer du texte à la façon de Guy de Maupassant. Pour se faire, j'ai utilisé le livre Bel Ami comme base de données. 
Parfois le texte généré est..plutôt surprenant! (mais amusant)

Le notebook a été exécuté sur Google Colab (25 Go de Ram et GPU Tesla P100). L'
entraînement du modèle s'est effectué en environ 1h.



In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
import numpy as np
import re
import io
from os import path

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importation et preprocessing :

In [3]:
file = open('/content/drive/MyDrive/Colab Notebooks/Projets/NLP_Text_Génération/Maupassant.txt','r', encoding='UTF-8')#_complet
content = file.readlines()

In [4]:
content[:10]

['Quand la caissière lui eut rendu la monnaie de sa pièce de cent sous, Georges Duroy sortit du restaurant.\n',
 'Comme il portait beau par nature et par pose d’ancien sous-officier, il cambra sa taille, frisa sa moustache d’un geste militaire et familier, et jeta sur les dîneurs attardés un regard rapide et circulaire, un de ces regards de joli garçon, qui s’étendent comme des coups d’épervier.\n',
 'Les femmes avaient levé la tête vers lui, trois petites ouvrières, une maîtresse de musique entre deux âges, mal peignée, négligée, coiffée d’un chapeau toujours poussiéreux et vêtue toujours d’une robe de travers, et deux bourgeoises avec leurs maris, habituées de cette gargote à prix fixe.\n',
 'Lorsqu’il fut sur le trottoir, il demeura un instant immobile, se demandant ce qu’il allait faire. On était au 28 juin, et il lui restait juste en poche trois francs quarante pour finir le mois. Cela représentait deux dîners sans déjeuners, ou deux déjeuners sans dîners, au choix. Il réfléchit q

In [5]:
script = [part_text.strip() for part_text in content]

In [7]:
def step_preprocessing(corpus):
  df_step2 = list()
  for sentences in script:
    sentence = re.findall(r'[A-Za-z0-9êçù.éàè?!]+',sentences)
    liste_temp = []
    for word in sentence:
      word = word.lower()
      ponctuation = re.findall(r'[?!.]',word)
      mot_sans_ponctuation = re.findall(r'[^?!.]+',word)
      try:
        liste_temp.append(mot_sans_ponctuation[0])
      except:
        ...
      try:
        liste_temp.append(ponctuation[0])
      except:
        continue
    df_step2.append(liste_temp)
    df_step3 = list()
    for i in range(len(df_step2)):
      df_step3.append(' '.join(df_step2[i]))
  return df_step3
df_step3 = step_preprocessing(script)

In [8]:
df_step3[:10]

['quand la caissière lui eut rendu la monnaie de sa pièce de cent sous georges duroy sortit du restaurant .',
 'comme il portait beau par nature et par pose d ancien sous officier il cambra sa taille frisa sa moustache d un geste militaire et familier et jeta sur les d neurs attardés un regard rapide et circulaire un de ces regards de joli garçon qui s étendent comme des coups d épervier .',
 'les femmes avaient levé la tête vers lui trois petites ouvrières une ma tresse de musique entre deux ges mal peignée négligée coiffée d un chapeau toujours poussiéreux et vêtue toujours d une robe de travers et deux bourgeoises avec leurs maris habituées de cette gargote à prix fixe .',
 'lorsqu il fut sur le trottoir il demeura un instant immobile se demandant ce qu il allait faire . on était au 28 juin et il lui restait juste en poche trois francs quarante pour finir le mois . cela représentait deux d ners sans déjeuners ou deux déjeuners sans d ners au choix . il réfléchit que les repas du mat

In [9]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(df_step3)
inp_sequences[:10]

[[75, 4],
 [75, 4, 5423],
 [75, 4, 5423, 24],
 [75, 4, 5423, 24, 201],
 [75, 4, 5423, 24, 201, 2261],
 [75, 4, 5423, 24, 201, 2261, 4],
 [75, 4, 5423, 24, 201, 2261, 4, 1167],
 [75, 4, 5423, 24, 201, 2261, 4, 1167, 1],
 [75, 4, 5423, 24, 201, 2261, 4, 1167, 1, 35],
 [75, 4, 5423, 24, 201, 2261, 4, 1167, 1, 35, 475]]

In [10]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = 30
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = tf.keras.utils.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

# Construction du modèle :

In [50]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 300, input_length=input_len))
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 29, 300)           3174300   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10581)             1068681   
Total params: 4,403,381
Trainable params: 4,403,381
Non-trainable params: 0
_________________________________________________________________


In [51]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15)

In [52]:
model.fit(predictors, label, epochs=200,callbacks=earlystop)

Epoch 1/200
3394/3394 [==============================] - 26s 7ms/step - loss: 6.6355
Epoch 2/200
3394/3394 [==============================] - 26s 8ms/step - loss: 5.7619
Epoch 3/200
3394/3394 [==============================] - 26s 8ms/step - loss: 5.2254
Epoch 4/200
3394/3394 [==============================] - 26s 8ms/step - loss: 4.8041
Epoch 5/200
3394/3394 [==============================] - 26s 8ms/step - loss: 4.4370
Epoch 6/200
3394/3394 [==============================] - 26s 8ms/step - loss: 4.1104
Epoch 7/200
3394/3394 [==============================] - 26s 8ms/step - loss: 3.8069
Epoch 8/200
3394/3394 [==============================] - 26s 8ms/step - loss: 3.5313
Epoch 9/200
3394/3394 [==============================] - 26s 8ms/step - loss: 3.2793
Epoch 10/200
3394/3394 [==============================] - 26s 8ms/step - loss: 3.0521
Epoch 11/200
3394/3394 [==============================] - 26s 8ms/step - loss: 2.8522
Epoch 12/200
3394/3394 [==============================] - 26s 8

In [ ]:
!mkdir -p saved_model
model.save('/content/drive/MyDrive/Colab Notebooks/Projets/NLP_Text_Génération/model_embed_300_final')

# Embedding Projector

Utilisation du site [Embedding Projector](https://projector.tensorflow.org/) de Tensorflow afin de visualiser le word embedding.

In [89]:
weights = model.layers[0].get_weights()[0]

In [90]:
out_v = io.open(path.join('/content', 'vecs.tsv'), 'w', encoding='utf-8')
out_m = io.open(path.join('/content', 'meta.tsv'), 'w', encoding='utf-8')

k = 0

for word, token in tokenizer.word_index.items():
    if k != 0:
        out_m.write('\n')
        out_v.write('\n')
    
    out_v.write('\t'.join([str(x) for x in weights[token]]))
    out_m.write(word)
    k += 1
    
out_v.close()
out_m.close()



<br>



<p align="center">
  <img width="500" height="400" src="https://github.com/AxelDucamp/PROJET_Text_Generation_Maupassant/blob/main/Embedding_Projector.png?raw=true">
</p>


<br>
<br>

# Résultats

In [16]:
def generate_text(seed_text, next_words, model, max_sequence_len,iter):
    output_temp = ""
    for iteration in range(iter):
      if iteration == 0:
        for _ in range(next_words):
            token_list = tokenizer.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
            predicted = np.argmax(model.predict(token_list, verbose=0))
            
            output_word = ""
            for word,index in tokenizer.word_index.items():
                if index == predicted:
                    output_word = word
                    break
            seed_text += " "+output_word
            output_temp = seed_text.title()
      else:
        for _ in range(next_words):
          text_suite = " ".join(output_temp.split()[-3:])
          token_list = tokenizer.texts_to_sequences([text_suite])[0]
          token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
          predicted = np.argmax(model.predict(token_list, verbose=0))
          
          output_word = ""
          for word,index in tokenizer.word_index.items():
              if index == predicted:
                  output_word = word
                  break
          output_temp += " "+output_word
    return output_temp.title()

In [55]:
print (generate_text("Il", 2, model, max_sequence_len,iter=5))
print (generate_text("Elle", 2, model, max_sequence_len,iter=5))

print (generate_text("La", 2, model, max_sequence_len,iter=5))
print (generate_text("Le", 2, model, max_sequence_len,iter=5))

Il Répondit Avec Calme Vous Avez D Abord Chercher Les Visages
Elle Se Leva Et Il Alla Vers Le Temps Elle Se
La Jeune Femme Répondit Oui Je Veux Bien Bête De Ne
Le Père Walter Le Regardait De Tout Seul L Il Revint


In [35]:
print (generate_text("Il allait", 2, model, max_sequence_len,iter=10))
print (generate_text("Elle sentait", 2, model, max_sequence_len,iter=10))

Il Allait Les Arracher Tout À Coup Il Eut Peur De La Trinité Murmura Oui Je Vous Aime Comme Ma Petite Fille
Elle Sentait Vaguement Qu Il S Approcha D Un Regard Rapide Était Grand Glace Des Salons Où Étaient Là Dans Le Visage


In [77]:
print (generate_text("J ai", 2, model, max_sequence_len,iter=8))
print (generate_text("Tu es", 2, model, max_sequence_len,iter=8))
print (generate_text("Quand", 2, model, max_sequence_len,iter=8))
print (generate_text("Quoi", 2, model, max_sequence_len,iter=8))

J Ai Un Peu Surpris Le Magistrat Demanda Encore Qu Est Ce Que Vous Avez Une Expérience Qui
Tu Es Méchante Suzette Ce Monsieur Dit Il Tu Vas Attraper Une Pauvre Femme Déchirée Par Une Inavouable
Quand Il Eut Fini Par Le F Reste Depuis Quelques Minutes D Tat Que Tu As Couché
Quoi Donc S Étant Assis Sur Un Banc Il S En Voulut De Temps En Temps Elle


In [83]:
print (generate_text("Pourquoi", 2, model, max_sequence_len,iter=2))
print (generate_text("Alors qu il", 2, model, max_sequence_len,iter=2))
print (generate_text("Alors qu elle", 2, model, max_sequence_len,iter=2))

Pourquoi Ça N Est Pas
Alors Qu Il Ne L Avait Pas
Alors Qu Elle S Approcha Un Peu


In [84]:
print (generate_text("Son amie", 2, model, max_sequence_len,iter=10))

Son Amie Mme Forestier Ne Remuait Point Elle Cria M Laroche Mathieu L Attendait En Montant L Escalier Il Se Releva Toute
